<a href="https://colab.research.google.com/github/Vaibhav-Ganatra/CNN/blob/master/MNIST/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0),(1))])
trainset = torchvision.datasets.MNIST(root="./Downloads",train=True,transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=2)

classes = (0,1,2,3,4,5,6,7,8,9)
num_classes = 10

In [ ]:
import torch.nn as nn
import torch.nn.functional as functions

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1,6,5,padding=2)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = self.pool(functions.relu(self.conv1(x)))
    x = self.pool(functions.relu(self.conv2(x)))
    x = x.view(-1,16*5*5)
    x = functions.relu(self.fc1(x))
    x = functions.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [ ]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
print(optimizer)

In [ ]:
print("Training started! Model learning ...")

for epoch in range(5):
  running_loss = 0.0

  for i,data in enumerate(train_loader,0):
    inputs, labels = data
    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
      print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
      running_loss = 0.0

print("Training Complete!")

Training started! Model learning ...
[1,  2000] loss: 0.242
[2,  2000] loss: 0.056
[3,  2000] loss: 0.042
[4,  2000] loss: 0.030
[5,  2000] loss: 0.024
Training Complete!


In [ ]:
testset = torchvision.datasets.MNIST(root='./Downloads', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=2)

In [ ]:
PATH = './mnist_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

n_net = Net()
n_net.load_state_dict(torch.load(PATH))

outputs = net(images)

# Making predictions for a few images
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(16)))

In [ ]:
# Testing the model on the Test set

correct = 0
total = 0
with torch.no_grad():
    i=0  
    for data in testloader:
        i+=16
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on '+ str(i)+' test images: %d %%' % (
    100 * correct /float(total)))

Accuracy of the network on 625 test images: 98 %


In [ ]:
# Here various Augmentation techniques such as RandomCrop, Adding Random Noise are present. 
# To test, we may use any combination of these transforms to check their effect on the accuracy of the model

transform_aug = transforms.Compose([
                              #  transforms.RandomCrop(28, padding=(5,5)),
                                transforms.ToTensor(),
                                transforms.Normalize((0),(1)),
                                # transforms.RandomHorizontalFlip(p=1),
                                # transforms.RandomVerticalFlip(1),
                                # AddRandomNoise()
                                ])
testset_aug = torchvision.datasets.MNIST(root='./Downloads', train=False,
                                       download=True, transform=transform_aug)
testloader_aug = torch.utils.data.DataLoader(testset_aug, batch_size=16,
                                         shuffle=False, num_workers=2)

In [ ]:
# Testing the model's performance on augmented datasets

correct = 0
total = 0
with torch.no_grad():
    i=0  
    for data in testloader_aug:
        i+=16
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on '+ str(i)+' test images with added augmentation: %d %%' % (
    100 * correct /float(total)))

Accuracy of the network on 10000 test images with added augmentation: 39 %


In [ ]:
# Function to add random noise to the image

class AddRandomNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size())*0.25